In [ ]:
%pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 38.5 MB/s eta 0:00:00


[자연어처리] 텍스트 분류 모델 구현


- 데이터셋 : 200개 한국어 뉴스 기사
- 라 벨 : 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)

In [1]:
import pandas as pd
import torch
import os
import string
import re
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from konlpy.tag import Okt
from collections import Counter
import numpy as np
from torch import optim
from torchmetrics.classification import MulticlassF1Score

In [5]:
for file in os.listdir(common_path):
    print(file)

0
1
2
2016-10-20.txt
3
4
5
6
7
corpus_data.txt


In [6]:
common_path='../data/Data/'


feature=[]
label=[]

for file in os.listdir(common_path):
    for file2 in os.listdir(common_path+file):
        with open(common_path+file+'/'+file2, 'r') as f:
            feature.append(f.read())
            label.append(file)

In [7]:
len(feature)

1600

In [8]:
news_df=pd.DataFrame({'feature':feature, 'label':label})

In [9]:
news_df

,feature,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문...",0
1,"예결위, 추경 막바지 심사 진통…여야 충돌\t(서울=연합뉴스) 김남권 기자 = 국회...",0
2,외압 논란·항명 사태…산 넘고 물 건넌 권성동 영장 청구\t안미현 검사 외압 폭로 ...,0
3,"친문 홍영표, 문빠에 찍혔다…특검 합의에 문자폭탄 공격\t대표적인 친(親)문재인계인...",0
4,"北, 연일 南비난…韓美정상회담 전 경고성 메시지 발신\t南, 맥스선더·태영호 등 불...",0
...,...,...
1595,"단일팀 추진' 대한카누연맹, 데상트코리아와 5년 후원 협약\t[스포티비뉴스=조형애 ...",7
1596,"올림픽 성공 뒷이야기... 서울대, 16일 이희범 평창 조직위원장 초청 특강\t[O...",7
1597,21일 개막 호치민3쿠션월드컵에 149명 참가 확정\t韓선수 46명 참가…1차 예선...,7
1598,"스포츠안전재단, 대축전에 안전필요성 알려\t[스타뉴스 채준 기자]\n\n\n스포츠안...",7


In [10]:
news_df['label'].value_counts()

label
0    200
1    200
2    200
3    200
4    200
5    200
6    200
7    200
Name: count, dtype: int64

In [11]:
news_df=news_df.sort_values('label')
news_df

,feature,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문...",0
127,"경찰, 김성태 폭행범 14일 기소의견으로 검찰 송치\t조직적 정치테러 가능성은 낮은...",0
128,"6.13재보궐 8곳이냐 12곳이냐…14일 결정된다\t국회의장·범진보 ""국민 참정권 ...",0
129,靑 “권력형적폐 청산 넘어 생활적폐 청산으로”\t청와대는 13일 문재인 정부의 1번...,0
130,"[지방선거 여론조사①] 서울·인천·경기, 민주당 후보 우세\t[앵커]\n\n6.13...",0
...,...,...
1469,신태용호 승선' 이승우 깜짝 귀국…21일 대표팀 합류\t2018 러시아 월드컵을 준...,7
1470,"2개 작품 준비한 피겨 최다빈 ""새로운 모습 보여주고파""\t[STN스포츠(목동)=이...",7
1471,"복싱 신종훈, 러시아 국제대회 결승 진출\t[한겨레] 키르기스탄 미를란 5-0 판정...",7
1461,단거리 최강마를 가려라! 20일 스포츠경향배\t【서울=뉴시스】김정환 기자 = 1등급...,7


In [12]:
news_df=news_df.reset_index(drop=True)
news_df

,feature,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문...",0
1,"경찰, 김성태 폭행범 14일 기소의견으로 검찰 송치\t조직적 정치테러 가능성은 낮은...",0
2,"6.13재보궐 8곳이냐 12곳이냐…14일 결정된다\t국회의장·범진보 ""국민 참정권 ...",0
3,靑 “권력형적폐 청산 넘어 생활적폐 청산으로”\t청와대는 13일 문재인 정부의 1번...,0
4,"[지방선거 여론조사①] 서울·인천·경기, 민주당 후보 우세\t[앵커]\n\n6.13...",0
...,...,...
1595,신태용호 승선' 이승우 깜짝 귀국…21일 대표팀 합류\t2018 러시아 월드컵을 준...,7
1596,"2개 작품 준비한 피겨 최다빈 ""새로운 모습 보여주고파""\t[STN스포츠(목동)=이...",7
1597,"복싱 신종훈, 러시아 국제대회 결승 진출\t[한겨레] 키르기스탄 미를란 5-0 판정...",7
1598,단거리 최강마를 가려라! 20일 스포츠경향배\t【서울=뉴시스】김정환 기자 = 1등급...,7


In [13]:
news_df['feature']=news_df['feature'].str.replace('\n', '')
news_df

,feature,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목\t최 부상, 행선지·방문...",0
1,"경찰, 김성태 폭행범 14일 기소의견으로 검찰 송치\t조직적 정치테러 가능성은 낮은...",0
2,"6.13재보궐 8곳이냐 12곳이냐…14일 결정된다\t국회의장·범진보 ""국민 참정권 ...",0
3,靑 “권력형적폐 청산 넘어 생활적폐 청산으로”\t청와대는 13일 문재인 정부의 1번...,0
4,"[지방선거 여론조사①] 서울·인천·경기, 민주당 후보 우세\t[앵커]6.13 지방선...",0
...,...,...
1595,신태용호 승선' 이승우 깜짝 귀국…21일 대표팀 합류\t2018 러시아 월드컵을 준...,7
1596,"2개 작품 준비한 피겨 최다빈 ""새로운 모습 보여주고파""\t[STN스포츠(목동)=이...",7
1597,"복싱 신종훈, 러시아 국제대회 결승 진출\t[한겨레] 키르기스탄 미를란 5-0 판정...",7
1598,단거리 최강마를 가려라! 20일 스포츠경향배\t【서울=뉴시스】김정환 기자 = 1등급...,7


In [14]:
news_df['feature']=news_df['feature'].str.replace('\t', '')
news_df

,feature,label
0,"동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목최 부상, 행선지·방문 목...",0
1,"경찰, 김성태 폭행범 14일 기소의견으로 검찰 송치조직적 정치테러 가능성은 낮은 듯...",0
2,"6.13재보궐 8곳이냐 12곳이냐…14일 결정된다국회의장·범진보 ""국민 참정권 위해...",0
3,靑 “권력형적폐 청산 넘어 생활적폐 청산으로”청와대는 13일 문재인 정부의 1번 국...,0
4,"[지방선거 여론조사①] 서울·인천·경기, 민주당 후보 우세[앵커]6.13 지방선거가...",0
...,...,...
1595,신태용호 승선' 이승우 깜짝 귀국…21일 대표팀 합류2018 러시아 월드컵을 준비하...,7
1596,"2개 작품 준비한 피겨 최다빈 ""새로운 모습 보여주고파""[STN스포츠(목동)=이보미...",7
1597,"복싱 신종훈, 러시아 국제대회 결승 진출[한겨레] 키르기스탄 미를란 5-0 판정으로...",7
1598,단거리 최강마를 가려라! 20일 스포츠경향배【서울=뉴시스】김정환 기자 = 1등급 경...,7


In [15]:
punc=string.punctuation

for p in punc:
    news_df['feature'] = news_df['feature'].str.replace(p, '')

news_df

,feature,label
0,동남아 담당 北 최희철 부상 베이징 도착…싱가포르행 주목최 부상 행선지·방문 목적 ...,0
1,경찰 김성태 폭행범 14일 기소의견으로 검찰 송치조직적 정치테러 가능성은 낮은 듯【...,0
2,613재보궐 8곳이냐 12곳이냐…14일 결정된다국회의장·범진보 국민 참정권 위해 1...,0
3,靑 “권력형적폐 청산 넘어 생활적폐 청산으로”청와대는 13일 문재인 정부의 1번 국...,0
4,지방선거 여론조사① 서울·인천·경기 민주당 후보 우세앵커613 지방선거가 이제 한달...,0
...,...,...
1595,신태용호 승선 이승우 깜짝 귀국…21일 대표팀 합류2018 러시아 월드컵을 준비하는...,7
1596,2개 작품 준비한 피겨 최다빈 새로운 모습 보여주고파STN스포츠목동이보미 기자새로운...,7
1597,복싱 신종훈 러시아 국제대회 결승 진출한겨레 키르기스탄 미를란 50 판정으로 꺾어’...,7
1598,단거리 최강마를 가려라 20일 스포츠경향배【서울뉴시스】김정환 기자 1등급 경주마들...,7


In [16]:
m=re.compile('[^ ㄱ-ㅣ가-힣]+')

news_df['feature']=news_df['feature'].apply(lambda x: m.sub(' ', x))
news_df

,feature,label
0,동남아 담당 최희철 부상 베이징 도착 싱가포르행 주목최 부상 행선지 방문 목적 ...,0
1,경찰 김성태 폭행범 일 기소의견으로 검찰 송치조직적 정치테러 가능성은 낮은 듯 서...,0
2,재보궐 곳이냐 곳이냐 일 결정된다국회의장 범진보 국민 참정권 위해 일 본회의...,0
3,권력형적폐 청산 넘어 생활적폐 청산으로 청와대는 일 문재인 정부의 번 국정...,0
4,지방선거 여론조사 서울 인천 경기 민주당 후보 우세앵커 지방선거가 이제 한달여 ...,0
...,...,...
1595,신태용호 승선 이승우 깜짝 귀국 일 대표팀 합류 러시아 월드컵을 준비하는 신태용호...,7
1596,개 작품 준비한 피겨 최다빈 새로운 모습 보여주고파 스포츠목동이보미 기자새로운 모...,7
1597,복싱 신종훈 러시아 국제대회 결승 진출한겨레 키르기스탄 미를란 판정으로 꺾어 억...,7
1598,단거리 최강마를 가려라 일 스포츠경향배 서울뉴시스 김정환 기자 등급 경주마들이...,7


In [17]:
def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens

    for token, count in counter.most_common(n_vocab):
        vocab.append(token)

    return vocab

In [18]:
train=news_df.sample(frac=0.9, random_state=81)
test=news_df.drop(train.index)

In [19]:
train.shape, test.shape

((1440, 2), (160, 2))

In [20]:
train.head()

,feature,label
1452,렛츠런파크 유소년 승마단 전국소년체육대회 참가한국마사회 렛츠런파크 부산경남본부장 정...,7
968,투병 매케인에 어차피 죽을사람 조롱한 백악관 보좌관 뭇매사악 혐오스럽다 나라위해 고...,4
1167,예약자 만 몰린 배틀그라운드 모바일 오늘 정식 출시서울뉴스 이수호 기자 지난해...,5
201,이란핵협정 탈퇴에도급락한 국제유가 왜뉴욕이데일리 이준기 특파원 국제유가가 일현...,1
1564,서울시봅슬레이스켈레톤연맹 회장배 대회강습회 개최마이데일리 윤욱재 기자 년 서울시...,7


In [24]:
stop_word='./stopwords.txt'

with open(stop_word, 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f]

In [25]:
tokenizer = Okt()

train_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in train['feature']]
test_tokens = [[token for token in tokenizer.morphs(text) if token not in stop_words] for text in test['feature']]

vocab = build_vocab(corpus=train_tokens, n_vocab=20000, special_tokens=['<pad>','<unk>'])
token_to_id = {token:idx for idx, token in enumerate(vocab)}
id_to_token = {idx:token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '하는', '기자', '있는', '명', '한국', '서울', '위', '이라고']
20002


In [26]:
def pad_sequences(sequences, max_length, pad_value):
    result = list()
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [27]:
unk_id = token_to_id["<unk>"]
train_ids = [[token_to_id.get(token, unk_id) for token in review] for review in train_tokens]
test_ids = [[token_to_id.get(token, unk_id) for token in review] for review in test_tokens]

max_length = 500
pad_id = token_to_id['<pad>']
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[ 3855  2383  1578  2917   212  1578  2384   532     6  2224  3855  2383
   342   732  3176 11606  1578  2917   185   156  1181   222    45   212
  1578  2384   532   342   732  1578  2917   156  3855  2383     7  2917
  1648  8686  1578  3307  2573  2917   811   387    13   387   109 19561
 19562  6877     6  2224 11607  2917 14335  1203   490   387   250  1471
   387   585   533   448   333   824   333  2917   251   440   490   414
   674   109   578     6  2224  2687  1047   634   303  1578  3307   387
   181    35  2225 19563   455   906   129     4  2917  1578   455  9894
     2  3177    97    40  3855  2383   342   732    36 19564   432   290
   227    59  2917  1323   324  1578  3307  1541   811   387   342   732
    68  1578  2917  1704  7706   569  1471 11606  3176   110   347   591
  1896 14336  2082   522  7707   560  3177   181   754  1304  2917  2303
   893 11608   324   825 19565   353  2688   347   774  2918  3651   474
   374   200   516  6266   324  3177  7708     0   

In [28]:
train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train['label'].values.astype(np.float32))
test_labels = torch.tensor(test['label'].values.astype(np.float32))

train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [29]:
n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [30]:
class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional=True, model_type = "lstm"):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim, padding_idx=0)

        if model_type == 'rnn':
            self.model = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        elif model_type == 'lstm':
            self.model = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim*2, 1)
        else :
            self.classifier = nn.Linear(hidden_dim, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output,_ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [42]:
epochs = 100
interval = 500
criterion=nn.CrossEntropyLoss()
model=SentenceClassifier(n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers).to(device)

HISTORY={'loss':[[],[]],'score':[[],[]]}

for epoch in range(epochs):
    model.train()
    loss_total=0
    score_total=0

    for feature,target in train_loader:
        feature=feature.to(device)
        target=target.to(device).long()

        pre_y=model(feature)
        loss=criterion(pre_y,target.reshape(-1))
        score=MulticlassF1Score(torch.softmax(pre_y,dim=1),target.reshape(-1).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total+=loss.item()
        score_total+=score.item()

    HISTORY['loss'][0].append(loss_total/len(train_loader))
    HISTORY['score'][0].append(score_total/len(train_loader))

    model.eval()
    with torch.no_grad():
        for feature,target in test_loader:
            feature=feature.to(device)
            target=target.to(device).long()

            val_pre_y=model(feature)

            loss=criterion(val_pre_y,target.reshape(-1))
            score=MulticlassF1Score(torch.softmax(val_pre_y,dim=1),target.reshape(-1).float())

        HISTORY['loss'][1].append(loss.item())
        HISTORY['score'][1].append(score.item())

TypeError: zeros() received an invalid combination of arguments - got (Tensor, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
